In [1]:
!pip install awscli

  Using cached awscli-1.25.76-py3-none-any.whl (3.9 MB)
  Using cached s3transfer-0.6.0-py3-none-any.whl (79 kB)
  Using cached colorama-0.4.4-py2.py3-none-any.whl (16 kB)
  Using cached PyYAML-5.4.1-cp39-cp39-win_amd64.whl (213 kB)
  Using cached rsa-4.7.2-py3-none-any.whl (34 kB)
  Using cached botocore-1.27.75-py3-none-any.whl (9.1 MB)
  Using cached docutils-0.16-py2.py3-none-any.whl (548 kB)
  Using cached python_dateutil-2.8.2-py2.py3-none-any.whl (247 kB)
  Using cached jmespath-1.0.1-py3-none-any.whl (20 kB)
  Using cached urllib3-1.26.12-py2.py3-none-any.whl (140 kB)
  Using cached pyasn1-0.4.8-py2.py3-none-any.whl (77 kB)
  Using cached six-1.16.0-py2.py3-none-any.whl (11 kB)


In [2]:
!pip install aws


  Using cached aws-0.2.5-py3-none-any.whl
  Using cached prettytable-3.4.1-py3-none-any.whl (26 kB)
  Using cached boto-2.49.0-py2.py3-none-any.whl (1.4 MB)
  Using cached fabric-2.7.1-py2.py3-none-any.whl (53 kB)
  Using cached paramiko-2.11.0-py2.py3-none-any.whl (212 kB)
  Using cached pathlib2-2.3.7.post1-py2.py3-none-any.whl (18 kB)
  Using cached invoke-1.7.1-py3-none-any.whl (215 kB)
  Using cached wcwidth-0.2.5-py2.py3-none-any.whl (30 kB)
  Using cached bcrypt-4.0.0-cp36-abi3-win_amd64.whl (153 kB)
  Using cached cryptography-38.0.1-cp36-abi3-win_amd64.whl (2.4 MB)
  Using cached PyNaCl-1.5.0-cp36-abi3-win_amd64.whl (212 kB)
  Using cached cffi-1.15.1-cp39-cp39-win_amd64.whl (179 kB)
  Using cached pycparser-2.21-py2.py3-none-any.whl (118 kB)


In [12]:
stream_name = 'blogpost-word-stream'
region = 'eu-west-1'
aws_profile = 'blogpost-kinesis'

In [4]:
!pip freeze 

aws==0.2.5
awscli==1.25.76
bcrypt==4.0.0
boto==2.49.0
botocore==1.27.75
certifi @ file:///C:/Windows/TEMP/abs_e9b7158a-aa56-4a5b-87b6-c00d295b01fanefpc8_o/croots/recipe/certifi_1655968940823/work/certifi
cffi==1.15.1
colorama==0.4.4
cryptography==38.0.1
docutils==0.16
fabric==2.7.1
invoke==1.7.1
jmespath==1.0.1
paramiko==2.11.0
pathlib2==2.3.7.post1
prettytable==3.4.1
pyasn1==0.4.8
pycparser==2.21
PyNaCl==1.5.0
python-dateutil==2.8.2
PyYAML==5.4.1
rsa==4.7.2
s3transfer==0.6.0
six==1.16.0
urllib3==1.26.12
wcwidth==0.2.5
wincertstore==0.2


In [28]:
import boto3
from boto.kinesis.exceptions import ResourceInUseException
import os
import time

if aws_profile:
    os.environ['AWS_PROFILE'] = aws_profile

# connect to the kinesis
# kinesis = boto.kinesis.connect_to_region(region)
kinesis = boto3.client('kinesis',
                      aws_access_key_id='qqqqq',
                      aws_secret_access_key='qqrrrrrrrrtttt'
                      )

def get_status():
    r = kinesis.describe_stream(StreamName=stream_name)
    description = r.get('StreamDescription')
    status = description.get('StreamStatus')
    return status

def create_stream(stream_name):
    try:
        # create the stream
        kinesis.create_stream(StreamName=stream_name, ShardCount=1)
        print('stream {} created in region {}'.format(stream_name, region))
    except ResourceInUseException:
        print('stream {} already exists in region {}'.format(stream_name, region))


    # wait for the stream to become active
    while get_status() != 'ACTIVE':
        time.sleep(1)
    print('stream {} is active'.format(stream_name))

In [29]:
create_stream(stream_name)

stream blogpost-word-stream created in region eu-west-1
stream blogpost-word-stream is active


In [34]:
import datetime
import time
import threading
from boto.kinesis.exceptions import ResourceNotFoundException

class KinesisProducer(threading.Thread):
    """Producer class for AWS Kinesis streams

    This class will emit records with the IP addresses as partition key and
    the emission timestamps as data"""

    def __init__(self, stream_name, sleep_interval=None, ip_addr='8.8.8.8'):
        self.stream_name = stream_name
        self.sleep_interval = sleep_interval
        self.ip_addr = ip_addr
        super().__init__()

    def put_record(self):
        """put a single record to the stream"""
        timestamp = datetime.datetime.utcnow()
        part_key = self.ip_addr
        data = timestamp.isoformat()

        kinesis.put_record(StreamName=self.stream_name, PartitionKey=part_key,Data=data)

    def run_continously(self):
        """put a record at regular intervals"""
        while True:
            self.put_record()
            time.sleep(self.sleep_interval)

    def run(self):
        """run the producer"""
        try:
            if self.sleep_interval:
                self.run_continously()
            else:
                self.put_record()
        except ResourceNotFoundException:
            print('stream {} not found. Exiting'.format(self.stream_name))

In [35]:
producer1 = KinesisProducer(stream_name, sleep_interval=2, ip_addr='8.8.8.8')
producer2 = KinesisProducer(stream_name, sleep_interval=5, ip_addr='8.8.8.9')
producer1.start()
producer2.start()

In [44]:
from boto.kinesis.exceptions import ProvisionedThroughputExceededException
import datetime

class KinesisConsumer:
    """Generic Consumer for Amazon Kinesis Streams"""
    def __init__(self, stream_name, shard_id, iterator_type,
                 worker_time=30, sleep_interval=0.5):

        self.stream_name = stream_name
        self.shard_id = str(shard_id)
        self.iterator_type = iterator_type
        self.worker_time = worker_time
        self.sleep_interval = sleep_interval

    def process_records(self, records):
        """the main logic of the Consumer that needs to be implemented"""
        raise NotImplementedError

    @staticmethod
    def iter_records(records):
        for record in records:
            part_key = record['PartitionKey']
            data = record['Data']
            yield part_key, data

    def run(self):
        """poll stream for new records and pass them to process_records method"""
        response = kinesis.get_shard_iterator(StreamName=self.stream_name,
            ShardId=self.shard_id, ShardIteratorType=self.iterator_type)

        next_iterator = response['ShardIterator']

        start = datetime.datetime.now()
        finish = start + datetime.timedelta(seconds=self.worker_time)

        while finish > datetime.datetime.now():
            try:
                response = kinesis.get_records(ShardIterator=next_iterator, Limit=25)

                records = response['Records']

                if records:
                    self.process_records(records)

                next_iterator = response['NextShardIterator']
                time.sleep(self.sleep_interval)
            except ProvisionedThroughputExceededException as ptee:
                time.sleep(1)

In [45]:
class EchoConsumer(KinesisConsumer):
    """Consumers that echos received data to standard output"""
    def process_records(self, records):
        """print the partion key and data of each incoming record"""
        for part_key, data in self.iter_records(records):
            print(part_key, ":", data)

In [46]:
shard_id = 'shardId-000000000000'
iterator_type = 'LATEST'
worker = EchoConsumer(stream_name, shard_id, iterator_type, worker_time=10)

In [47]:
worker.run()

8.8.8.8 : b'2022-09-18T20:48:10.550973'
8.8.8.8 : b'2022-09-18T20:48:12.668722'
8.8.8.9 : b'2022-09-18T20:48:12.793771'
8.8.8.8 : b'2022-09-18T20:48:14.789496'
8.8.8.8 : b'2022-09-18T20:48:16.915309'
8.8.8.9 : b'2022-09-18T20:48:17.918172'
8.8.8.8 : b'2022-09-18T20:48:19.039138'


In [48]:
kinesis.delete_stream(StreamName=stream_name)

{'ResponseMetadata': {'RequestId': 'e8704b3a-0048-b62a-b3d1-cdbb5fbda437',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'e8704b3a-0048-b62a-b3d1-cdbb5fbda437',
   'x-amz-id-2': 'XKQTxTpdJsLhjcav1EKUhRKwZ1JErM5UBBrAM2bLwcWzE6gXUq+cO4YE0a8qu916RGBeZM5AYvWiFj0b2f7aMblOY4ddcJCS',
   'date': 'Sun, 18 Sep 2022 20:49:05 GMT',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '0'},
  'RetryAttempts': 0}}

Exception in thread Thread-15:
Traceback (most recent call last):
  File "S:\anaconda\lib\threading.py", line 973, in _bootstrap_inner
    self.run()
  File "C:\Users\aadar\AppData\Local\Temp\ipykernel_20988\2317068655.py", line 36, in run
  File "C:\Users\aadar\AppData\Local\Temp\ipykernel_20988\2317068655.py", line 29, in run_continously
  File "C:\Users\aadar\AppData\Local\Temp\ipykernel_20988\2317068655.py", line 24, in put_record
  File "S:\anaconda\lib\site-packages\botocore\client.py", line 401, in _api_call
    return self._make_api_call(operation_name, kwargs)
  File "S:\anaconda\lib\site-packages\botocore\client.py", line 731, in _make_api_call
    raise error_class(parsed_response, operation_name)
botocore.errorfactory.ResourceNotFoundException: An error occurred (ResourceNotFoundException) when calling the PutRecord operation: Stream blogpost-word-stream under account 284677672991 not found.
Exception in thread Thread-16:
Traceback (most recent call last):
  File "S:\anacon